In [10]:
import cv2 as cv
import numpy as np
import mediapipe as mp
import pandas as pd
import ydf
import numpy as np
import os
import torch
import torch.nn.functional as F 
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers=nn.Sequential(
            nn.Linear(len(train.columns)-2,1024),
            nn.ReLU(),
            nn.Linear(1024,512),
            nn.ReLU(),
            nn.Linear(512,364),
            nn.ReLU(),
            nn.Linear(364,128),
            nn.ReLU(),
            nn.Linear(128,64),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(64,29),
            # nn.ReLU(),
            # nn.Linear(32,29)
        )
    def forward(self,x):
        logits=self.layers(x)

        return logits

In [6]:
from torch.utils.data import Dataset
class gestureCreator(Dataset):
    def __init__(self,dataframe):
        self.data=dataframe
    def __len__(self):
        return len(self.data)
    def __getitem__(self):
        features=self.data.values.astype(float) 
        features=torch.tensor(features,dtype=torch.float32)
        return features   

In [7]:
m=torch.load('model_f.pt')
m.eval()
device = torch.device('cpu')


In [11]:
m2=ydf.load_model("/Users/mohammadmendahawi/Desktop/JIT/model_tree")

In [12]:
mph=mp.solutions.hands
hand=mph.Hands()
mpdraw=mp.solutions.drawing_utils

I0000 00:00:1728336463.161443 3089366 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M2


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1728336463.173596 3092199 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1728336463.179370 3092199 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [13]:
gesture_mapper={
    
    0: 'A',
    1: 'B',
    2: 'C',
    3: 'D',
    4: 'del',
    5: 'E',
    6: 'F',
    7: 'G',
    8: 'H',
    9: 'I',
    10: 'J',
    11: 'K',
    12: 'L',
    13: 'M',
    14: 'N',
    15: 'nothing',
    16: 'O',
    17: 'P',
    18: 'Q',
    19: 'R',
    20: 'S',
    21: 'space',
    22: 'T',
    23: 'U',
    24: 'V',
    25: 'W',
    26: 'X',
    27: 'Y',
    28: 'Z'
}

In [14]:
columns=[f'{coord}{i}'for i in range(21) for coord in ['x','y','z']]

In [15]:
def control():
    cap=cv.VideoCapture(1)
    while True:
        ret,frame=cap.read()

        cv.flip(frame,1)
        frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        result = hand.process(frame_rgb)
        if result.multi_hand_landmarks:
            for hand_index, hand_landmarks in enumerate(result.multi_hand_landmarks):
                handedness = result.multi_handedness[hand_index].classification[0].label
                mp_draw.draw_landmarks(frame, hand_landmarks, mp_h.HAND_CONNECTIONS)
                land = []
                land2=[hand_index]

                for lm in hand_landmarks.landmark:

                    land.extend([lm.x, lm.y, lm.z])

                    land2.extend([lm.x, lm.y, lm.z])

    

                df=pd.DataFrame([land2],columns=columns)

                pred=m2.predict(df)

                

                features=torch.tensor(land,dtype=torch.float32).unsqueeze(0).to(device)

                with torch.no_grad():

                    out=m(features)

                    _,val=torch.max(out,1)

                    pred=val.item()

                    label=gesture_mapping.get(pred,"Unknown")

                avg=(out.cpu().numpy()+pred)/2

                pred_ensemble=np.argmax(avg,axis=1)[0]

                label_ensemble=gesture_mapping.get(pred_ensemble,"Unkown")

                wrist_pos=hand_landmarks.landmark[0]

                h,w,_=frame.shape

                wrist_x=int(wrist_pos.x*w)

                wrist_y=int(wrist_pos.y*h)

                pos=(wrist_x,wrist_y-30)




                cv.putText(

                    frame, 

                    f'{label_ensemble}', 

                    pos, 

                    cv.FONT_HERSHEY_SIMPLEX, 

                    1, 

                    (255, 0, 0), 

                    2, 

                    cv.LINE_AA

                )

                tasks(label_ensemble)




        # Display the image 

        cv.imshow('feed', cv.flip(frame, 1))

        if cv.waitKey(1) == ord("q"):

            cap.release()




            cv.destroyAllWindows()

            break

In [16]:
control

<function __main__.control()>